<a href="https://colab.research.google.com/github/anmutechsupport/enigmaXbombe/blob/main/enigmaXbombe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is the Enigma and Bombe

The Enigma is electromechanical cryptographic machine that the German military used to encode and protect strategic messages during WW2. 

Most infamously, the Enigma machine was used to coordinate "wolf-pack" U-boat attacks in the Battle of the Atlantic. These attacks, alongside U-boat blockades in the Atlantic enaabled by the Enigma slowed down and prevented imports from naval trade with the US and Canada. This imports were crucial to Britain's sustainability in the war. 


---


The turing-Welchman Bombe was an electromechanical machine that aimed crack the Enigma settings to decode intercepted messages. 

With the development of the Turing-Welchman Bombe at Bletchley park, Allies intelligence agencies were able to crack the Enigma settings in 2-4 hours, which enabled for decrypting 3000-5000 messages to be decrypted daily, saving countless lives. Because of the decryption scale provided by the Bombe machine, it is believed that the war was shortened by more than 2 years.





# How is the Enigma used?

![](https://drive.google.com/uc?export=view&id=1whGgoZhaw3EBfli5Uuoi4e5D18gxZ4wB)

At first glance, the Enigma looks like any other typewriter, but in reality it's a complex system that involves a ton of moving parts to hide the true message of the operator. More specifically, when an Enigma operator presses on a key, an electrical signal passes through a complex web of wiring, controlled by the plugboard and rotor, which leads to one of the letters on the lampboard to flash. 

The flashed letters would be noted and sent in morse code to a reciever over radio. The reciever would take the encoded message and pass it through their own Enigma with the exact same rotor and plugboard settings as the sender. Because of the reciprocal nature of the Enigma, the lampboard would flash the original plaintext to the reciever. 

![](https://drive.google.com/uc?export=view&id=1witPeZqsN_EqMwXI6HXdlDL1Aiawu0dO)

The rotor and plugboard settings would change everyday. These settings would be known across Enigma operators through a code book that would include pages like the one above. Here's a breakdown of the columns:

1. **Datum = Date:** the date's bottom to top because the operators would rip off the settings for each day from the book to ensure security
2. **Walzenlage = Rotor Type and Order**: Operators had 5 different rotors to choose from and could place them in any order. Each rotor had it's own unique wiring and turning point. 
3. **Ringstellung = Ring Setting:** Each rottor had a ring setting which would offset it's wiring relative to the alphabet. For example [A, B, C] -> [F, H, D] mapping shifted by 2 ring settings would become [A, B, C] -> [H, J, F]. As you can see, each encoding shifted 2 places up in the Alphabet. 
4. **Steckerverbindungen = Plugboard:** Before a letter passes through the rotors, it is first swapped by another letter, a mapping that is explicitly set by the operator. For example, if A is "steckered" to B, when an operator clicked on A, the electrical signal would pass through the usual B path, and vice versa.
5. **Kenngruppen = Date Discriminant**: To specify which date's Enigma setting a sender was using, they would send the date discriminant as morse code over radio.

Once an operator configures their Enigma for the day, they would be ready to both send and recieve messages. 

Here's the step-by-step for sending a message with the Enigma:
1. Operator thinks of 3 letters at random, which will be used as the Grundtellung or initial rotor position of the 3 rotors. E.g. [F, E, G] can be used as the initial positions. 
2. Operator thinks of another 3 random letters which would be used for the message-setting. These letters would then be typed into the Enigma, and the outputs would be written down. The rotor positions would then be turned to the new 3 letters. E.g [A, B, C] -> [F, E, G] -> [Z, X, W]
3. The intended message would then be typed into the Enigma with the new settings. Over radio transmission in morse code, standard information, the Kenngruppe discriminant and the Grundstellung would be broadcasted without encryption. The second part of the message would include the enciphered message-setting with the full enciphered message. 

Next, here's the step-by-step for deciphering an Enigma message:
1. Operator would use the Grundstellung rotor position to first decipher the enciphered message-setting (in our example, it is [Z, X, W]), which would output the necessary rotor position to decipher the rest of the message (which would be [A, B, C])
2. After setting the rotor positions to the message-setting, the operator then types in the rest of the message, which is returned as plaintext on the lampboard. 

Deciphering is easy with the Enigma because of it's reciprocity. If you use the same Enigma core settings, you are able to both encipher and decipher the message the same. 


# The inner workings of the Enigma

![](https://drive.google.com/uc?export=view&id=1_J8yVQVLNIzn6cE_YSjjmK2ArCvY-Fuo)

The diagram above showcases how the signal of a single keypress passes through the different components of the Enigma. The diagram already looks crazy, but the most intimidating factor of the Enigma is that the rotor paths change every key press, meaning this diagram would look different if the key G was pressed a second time.   

Just like how the minute handle on a clock changes position once the second handle makes a full rotation, the rotors are organized in a way that whenever the rotor to the right of each rotor reached it's turning point, it would shift by one position. The right-most rotor would shift every keypress, the center rotor would shift everytime the right-most rotor reached it's turning points, and the last rotor would shift whenever the center rotor reached it's turning point. 

The main purpose of this system was to remove the simplicity of substitution-based encryption so that a single letter wouldn't be encoded to the same letter in a pattern. For example, if you pressed the A key twice, it would be encoded differently each time. 

If that was a lot to take in, don't worry, this notebook will be breaking down each component of the Enigma using a python simulation. 


### Setting up

In [ ]:
from collections import defaultdict
import os

ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
N_LETTERS = 26

## Reflectors

In [ ]:
class Reflector:

    def __init__(self, name, letters):
        """
        Initial configuration of the reflector's data.
        """

        # name
        self.name = name

        # letters
        self.reflections = defaultdict()
        for pin_in, output_letter in enumerate(letters):
            pin_out = ALPHABET.index(output_letter)
            self.reflections[pin_in] = pin_out

    def reflect(self, pin_in):
        pin_out = self.reflections[pin_in]
        return pin_out


Starting with the most simple component of the Enigma, the reflector simply projects one letter onto another. Enigma operators had 4 main reflectors to choose from, named B, C, B thin, and C thin. Each reflector had a different mapping between letters. 

The Reflector class above takes in a name (e.g B) and a set of letters to be mapped relative to the Alphabet. To initialize the reflections, I loop through the set of letters and their indexes and map them to the index of the letter relative to the alphabet. The reflect function can then be called upon the Reflector so that the pin_in variable can be used to index the reflection mappings. 

In [ ]:
reflector_C = ['C', 'YRUHQSLDPXNGOKMIEBFZCWVJAT']

print("---Mapping---")
print(ALPHABET)
print(reflector_C[1])

reflector_C = Reflector(name=reflector_C[0], letters=reflector_C[1])

pin_in_A = ALPHABET.index('A')
pin_out_A = reflector_C.reflect(pin_in=pin_in_A)
out_A = ALPHABET[pin_out_A]

print("---testing---")
print("A maps to:", out_A)

---Mapping---
ABCDEFGHIJKLMNOPQRSTUVWXYZ
YRUHQSLDPXNGOKMIEBFZCWVJAT
---testing---
A maps to: Y


In the example above, I used the settings for the reflector C and outputted the reflection of letter A. In the mapping log, you can see that the letter A maps to the letter Y. I then initialized a Reflector object that holds the settings of reflector C. The reflect method was called on the pin index of the letter A (which would be 0) which outputted the pin index of the letter Y (which is 24).

## Rotors

In [ ]:
class Rotor:

    def __init__(self, name, letters, turnovers):
        """
        Initial configuration of the rotor's data.
        """

        # name
        self.name = name

        # letters
        self.wiring = defaultdict()
        self.wiring_inverse = defaultdict()
        for pin_in, output_letter in enumerate(letters):
            pin_out = ALPHABET.index(output_letter)
            self.wiring[pin_in] = pin_out
            self.wiring_inverse[pin_out] = pin_in

        # turnovers
        self.turnovers = []
        for turn in turnovers:
            pin_turn = ALPHABET.index(turn)
            self.turnovers.append(pin_turn)


    def load_config(self, ring_setting_letter, offset_letter):

        self.ring_setting = ALPHABET.index(ring_setting_letter) 
        self.position = ALPHABET.index(offset_letter)

        # correct for ring setting (position and turnovers) -> you use modulo N_LETTERS to offset for negative numbers
        self.position = (self.position - self.ring_setting) % N_LETTERS # measures position shifts for output contact -> you could use multiple combinations of init position and ring setting to get the same core position
        # print((self.position - self.ring_setting) % N_LETTERS)

        for i, turnover in enumerate(self.turnovers):
            new_turnover = (turnover - self.ring_setting) % 26
            self.turnovers[i] = new_turnover

    def step(self, turnover=False):
        """
        Returns true if the rotor to its left has to turnover.

        turnover = signal if this rotor turns
        """
        turn_left = False

        if turnover:
            self.position = ( self.position + 1 ) % N_LETTERS
            if self.position in self.turnovers:
                turn_left = True

        return turn_left


    def process_inwards(self, std_in):
        """
        Receives input signal and returns output pin (right to left)
        """
        pin_in = (std_in + self.position) % N_LETTERS # offseting pin input by rotor position + ring setting offset 
        
        pin_out = self.wiring[pin_in]
        pin_out = (pin_out - self.position) % N_LETTERS

        return pin_out


    def process_outwards(self, std_out):
        """
        Receives output signal and returns input pin (left to right)
        """
        pin_out = (std_out + self.position) % N_LETTERS
        
        pin_in = self.wiring_inverse[pin_out]
        pin_in = (pin_in - self.position) % N_LETTERS

        return pin_in



Now comes the more difficult component of the Enigma, the rotors. The `Rotor` class must have a few different properties:
- Have the ability to turn (obviously) 
- Have the ability to translate letters both inwards and outwards (important charecteristic for Enigma reciprocity)
- Have a turnover point(s) at which the rotor to the left of itself turns
- Have the ability to shift its letter mapping (wiring) based on its ring settings and rotor position (Grundstellung or message-setting)

The `Rotor` class above first initializes with a name, a set of letters to be mapped to, and turnover letters. The rotor is fully initialized once the `load_config` function is called, in which the ring setting, rotor position, and final core position is set. The main logic behind the `position` property is that core position of the rotor is offset by the difference in positions between the ring setting and rotor position. E.g. if a rotor has an initial position of G and the ring setting is 2, the total offset from the original wiring would be the index of G in the alphabet (which is 4) minus the ring setting, which would be 2. 

The `Rotor` class also comes with a `step` method, which spins the rotor by one position in the case that the `turnover` is true. The method also returns boolean value that indicates whether the rotor to the left of it should also step, based on the initialized turnovers of the rotor. 

Lastly, there are 2 methods that used for translating letters through the rotor: `process_inwards` and `process_outwards`. In these methods we see the `position` property comes into play; we offset the input pin by adding the position value to it. The new offseted pin input then indexes the wiring lists that were initialized with the rotor. The pin output from the wiring list is then offset once again by the `position` parameter, this time subtracting the value from the pin output. The `wiring_inverse` is the reverse of the original rotor mappings, which is used for outward letter translations. 



In [ ]:
rotor_I = ['I','EKMFLGDQVZNTOWYHXUSPAIBRCJ','R']
core_position = ['A', 'A']
print("---Mapping---")
print(ALPHABET)
print(rotor_I[1])

rotor_I = Rotor(name=rotor_I[0], letters=rotor_I[1], turnovers=rotor_I[2])
rotor_I.load_config(ring_setting_letter=core_position[0], offset_letter=core_position[1])

pin_in_D = ALPHABET.index('D')
forward_D = rotor_I.process_inwards(std_in=pin_in_D)
out_D = ALPHABET[forward_D]

print("---testing---")
print("D maps to:", out_D)

---Mapping---
ABCDEFGHIJKLMNOPQRSTUVWXYZ
EKMFLGDQVZNTOWYHXUSPAIBRCJ
---testing---
D maps to: F


In the example above, I instantiate a rotor object which represents Rotor I, one of the 5 rotors used during WW2. The output above looks similar to that off the reflector, because the ring setting and rotor position I used don't add any offset to the rotor. 

In [ ]:
rotor_I_settings = ['I','EKMFLGDQVZNTOWYHXUSPAIBRCJ','R']
core_position = ['A', 'B']
print("---Mapping---")
print(ALPHABET)
print(rotor_I_settings[1])

rotor_I = Rotor(name=rotor_I_settings[0], letters=rotor_I_settings[1], turnovers=rotor_I_settings[2])
rotor_I.load_config(ring_setting_letter=core_position[0], offset_letter=core_position[1])

pin_in_D = ALPHABET.index('D')
forward_D = rotor_I.process_inwards(std_in=pin_in_D)
out_D = ALPHABET[forward_D]

print("---testing---")
print("D maps to:", out_D)
# print(rotor_I.position)

---Mapping---
ABCDEFGHIJKLMNOPQRSTUVWXYZ
EKMFLGDQVZNTOWYHXUSPAIBRCJ
---testing---
D maps to: K


This time, when we change the core settings by shifting the rotor by one position, our output gets shifted by 2 places. D maps to K, which makes sense given that:
1. The letter D is shifted by +1 relative to the Alphabet to E
2. E maps to L 
3. L shifts by -1 relative to the Alphabet to K


In [ ]:
print("---Mapping---")
print(ALPHABET)
print(rotor_I_settings[1])

rotor_I.step(turnover=True)

pin_in_D = ALPHABET.index('D')
forward_D = rotor_I.process_inwards(std_in=pin_in_D)
out_D = ALPHABET[forward_D]

print("---testing---")
print("D maps to:", out_D)
# print(rotor_I.position)

---Mapping---
ABCDEFGHIJKLMNOPQRSTUVWXYZ
EKMFLGDQVZNTOWYHXUSPAIBRCJ
---testing---
D maps to: E


With the final example, I show how rotor steps alter the translation of the same letter. I'm using the exact same `rotor_I` with the initial rotor position offset of 1. Here's why D translates to E:     
1. Now that the rotor position has stepped once, our overall shift or `position` parameter changes from 1 to 2
2. The letter D is shifted by +2 relative to the Alphabet to F
3. F maps to G
4. G shifts by -2 relative to the Alphabet to E

## Assembling everything together

In [ ]:
#config format: name, letter map, turnover point
rotor_configs = [
                  ['I','EKMFLGDQVZNTOWYHXUSPAIBRCJ','R'], 
                  ['II','AJDKSIRUXBLHWTMCQGZNPYFVOE','F'], 
                  ['III','BDFHJLCPRTXVZNYEIWGAKMUSQO','W'], 
                  ['IV','ESOVPZJAYQUIRHXLNFTGKDCMWB','K'], 
                  ['V','VZBRGITYUPSDNHLXAWMJQOFECK','A'], 
]

#config format: name, letter map
reflector_configs = [
                      ['B','YRUHQSLDPXNGOKMIEBFZCWVJAT'],
                      ['C','FVPJIAOYEDRZXWGCTKUQSBNMHL'],
                      ['BT','ENKQAUYWJICOPBLMDXZVFTHRGS'],
                      ['CT','RDOBJNTKVEHMLFCWZAXGYIPSUQ'],
]

In [ ]:
class EnigmaMachine:

    def __init__(self):
        """
        Initial configuration of the machine.
        """

        # load data
        self.load_data()


    def load_data(self):
        
        # rotors
        self.rotors_data = defaultdict()
        for cfg in rotor_configs: 
            rotor_name, rotor_letters, rotor_turnovers = cfg
            rotor = Rotor(rotor_name, rotor_letters, rotor_turnovers)
            self.rotors_data[rotor_name] = rotor

        # reflectors
        self.reflectors_data = defaultdict()
        for cfg in reflector_configs: 
            reflector_name, reflector_letters = cfg
            reflector = Reflector(reflector_name, reflector_letters)
            self.reflectors_data[reflector_name] = reflector


    def load_config_dict(self, config_dict):

          # rotors
          self.n_rotors = config_dict['r_count']
          self.rotor_names = config_dict['rotors']

          rotor_ring_settings = config_dict['ring_settings']
          rotor_offsets = config_dict['rotor_positions']

          self.rotors = defaultdict()
          for i, rotor_name in enumerate(self.rotor_names):
              if rotor_name not in self.rotors_data: 
                  print("ERROR: There is no rotor with the name: {}".format(rotor_name))
                  exit()
              else: 
                  rotor = self.rotors_data[rotor_name]
                  rotor.load_config(rotor_ring_settings[i], rotor_offsets[i])
                  self.rotors[rotor_name] = rotor

          # plugboard
          self.n_plugboard_pairs = config_dict['n_plugboard_pairs']
          self.plugboard = defaultdict()
          if self.n_plugboard_pairs > 0: 
              plugboard_pairs = config_dict['plugboard_pairs']
              for pair in plugboard_pairs: 
                  l1, l2 = pair[0], pair[1]
                  self.plugboard[l1] = l2
                  self.plugboard[l2] = l1

          # reflector
          self.reflector_name = config_dict['reflector']
          if self.reflector_name not in self.reflectors_data: 
              print("ERROR: There is no reflector with the name: {}".format(self.reflector_name))
              exit()
          else: 
              self.reflector = self.reflectors_data[self.reflector_name]


    def load_config_bombe(self, n_rotors, rotor_names, rotor_ring_settings,
                           rotor_offsets, reflector_name):

        # rotors
        self.n_rotors = n_rotors
        self.rotor_names = rotor_names

        self.rotors = defaultdict()
        for i, rotor_name in enumerate(self.rotor_names):
            if rotor_name not in self.rotors_data: 
                print("ERROR: There is no rotor with the name: {}".format(rotor_name))
                exit()
            else: 
                rotor = self.rotors_data[rotor_name]
                rotor.load_config(rotor_ring_settings[i], rotor_offsets[i])
                self.rotors[rotor_name] = rotor

        # reflector
        self.reflector_name = reflector_name
        if self.reflector_name not in self.reflectors_data: 
            print("ERROR: There is no reflector with the name: {}".format(self.reflector_name))
            exit()
        else: 
            self.reflector = self.reflectors_data[self.reflector_name]


    def step_rotors(self):
        # the right rotor (last one in list) always rotates,
        # from there others rotate like a clock works
        # i.e. turn when the rotor to the right passes 
        #      through its turnover letter

        rotor_names_inverted = self.rotor_names[::-1]
        rotor_positions = []

        # most-right rotor
        right_rotor_name = rotor_names_inverted[0]
        right_rotor = self.rotors[right_rotor_name]
        turn_signal = right_rotor.step(turnover=True)
        rotor_positions.append(right_rotor.position)

        # the rest
        for rotor_name in rotor_names_inverted[1:]:
            rotor = self.rotors[rotor_name]
            turn_signal = rotor.step(turnover=turn_signal)
            rotor_positions.append(rotor.position)

        # update status
        self.rotor_positions = rotor_positions[::-1]


    def cipher_letter(self, letter):

        # advance rotors
        self.step_rotors()
        
        # ciphered letter
        ciphered_letter = ""
        
        # enter plugboard switch
        plugboard_letter = self.plugboard.get(letter, letter) # use get method on dict so that if a map does not exist, it will just return the letter

        # enter rotors (inwards - right to left)
        rotor_names_inverted = self.rotor_names[::-1]
        
        pin_in = ALPHABET.index(plugboard_letter)

        for rotor_name in rotor_names_inverted:
            rotor = self.rotors[rotor_name]
            pin_in = rotor.process_inwards(pin_in)

        # enter reflector
        pin_out = self.reflector.reflect(pin_in)

        # enter rotors (outwards - left to right)
        for rotor_name in self.rotor_names:
            rotor = self.rotors[rotor_name]
            pin_out = rotor.process_outwards(pin_out)

        # enter plugboard switch
        rotor_letter = ALPHABET[pin_out]
        plugboard_letter = self.plugboard.get(rotor_letter, rotor_letter)

        # calculate ciphered letter
        ciphered_letter = plugboard_letter

        '''
        print(ciphered_letter, 
              (self.rotors['V'].position + self.rotors['V'].ring_setting) % N_LETTERS, 
              (self.rotors['IV'].position + self.rotors['IV'].ring_setting) % N_LETTERS, 
              (self.rotors['I'].position + self.rotors['I'].ring_setting) % N_LETTERS)
        '''

        return ciphered_letter
    
    
    def cipher_letter_bombe(self, letter):

        # enter rotors (inwards - right to left)
        rotor_names_inverted = self.rotor_names[::-1]
        
        pin_in = ALPHABET.index(letter)

        for rotor_name in rotor_names_inverted:
            rotor = self.rotors[rotor_name]
            pin_in = rotor.process_inwards(pin_in)

        # enter reflector
        pin_out = self.reflector.reflect(pin_in)

        # enter rotors (outwards - left to right)
        for rotor_name in self.rotor_names:
            rotor = self.rotors[rotor_name]
            pin_out = rotor.process_outwards(pin_out)

        # enter plugboard switch
        rotor_letter = ALPHABET[pin_out]

        # calculate ciphered letter
        ciphered_letter = rotor_letter
        
        return ciphered_letter


    def cipher_text(self, text, print_display=True):

        ciphered_text = ""

        for letter in text:
            ciphered_letter = self.cipher_letter(letter)
            ciphered_text += ciphered_letter

            if print_display:
                print("ROTOR PANEL: \t", self.rotor_positions)
                print("CIPHER: \t {} -> {}\n".format(letter, ciphered_letter))

        return ciphered_text
    

Above is the code to instantiate a COMPLETE Enigma machine in python, which  incorporates the existing `Rotor` and `Reflector` classes with the addition of a plugboard. Starting off, we initialize the class by creating a set of parameter lists that hold configurations of rotors and reflectors that were used during WW2. The next component of initialization entails actually configuring the machine with the settings that an operator specifies. These settings ofcourse include:
- Rotor names and their order
- Ring settings and rotor positions for each rotor
- List of plugboard 
- Type of reflector



In [ ]:
enigma_config = {
    'r_count': 3,
    'rotors': ['V', 'IV', 'I'],
    'ring_settings': ['N', 'D', 'V'],
    'rotor_positions': ['D', 'Y', 'A'],
    'n_plugboard_pairs': 10,
    'plugboard_pairs': ['DI', 'ZL', 'RX', 'UH', 'QK', 'PC', 'VY', 'GA', 'SO', 'EM'],
    'reflector': 'C'
}

Above is an example of what a configuration object would look like. Taking a deeper look at the plugboard pairs, they are simply a set of letters that are manually configured to swap before they enter the rotors of the machine. In the code, the plugboard pairs are simply mapped to each other in a dictionary, similar to what we I did with the rotor and reflector. 

You might be thinking a lot of these individual components of the Enigma do the exact same thing: swapping one letter for another. **What makes the Enigma so complex aren't it's individual translations but the combination of encodings as text flows through the system.**


In [ ]:
ciphered_text = ""
for letter in letters:

    # advance rotors
    step_rotors()

    # enter plugboard switch
    plugboard_letter = plugboard.get(letter, letter) # use get method on dict so that if a map does not exist, it will just return the letter

    # enter rotors (inwards - right to left)
    rotor_names_inverted = rotor_names[::-1]

    pin_in = ALPHABET.index(plugboard_letter)

    for rotor_name in rotor_names_inverted:
        rotor = rotors[rotor_name]
        pin_in = rotor.process_inwards(pin_in)

    # enter reflector
    pin_out = reflector.reflect(pin_in)

    # enter rotors (outwards - left to right)
    for rotor_name in rotor_names:
        rotor = rotors[rotor_name]
        pin_out = rotor.process_outwards(pin_out)

    # enter plugboard switch
    rotor_letter = ALPHABET[pin_out]

    # calculate ciphered letter
    ciphered_letter = plugboard.get(rotor_letter, rotor_letter)
    ciphered_text += ciphered_letter



The most important method to understand in the `EnigmaMachine` is the `cipher_letter`, which passes a single letter through all the components. Above is the pseudo code for the method which showcases the main steps and translations: 

1. The `step_rotors()` changes the state of the rotors by one step before translations take place. The right rotor will always turn on each input, and depending on the turnover point, the other rotors may shift as well. 
2. The input letter is passed through the plugboard, and if there is a pair that includes the letter, the letter will be swapped. If not, the letter will remain the same.
3. For the forwardpass, the rotor order is reversed such that rotor on the rightmost is given the letter pin first. The letter pin then passes through each rotor from right to left, using the `process_inwards` method.
4. After passing through the rotors, the final input pin is translated by the reflector, which entails using the `reflect` method on the pin. This part is enables the reciprocity of Enigma. Because we have this "mirror" that sends the pin back to the rotors inversely, we can encipher and decipher messages using the same Enigma settings. 
5. The ouput pin returns to the rotors, now passing through them left-to-right with inverse wiring using the `process_outwards` method.
6. Finally, the output pin indexes the Alphabet and the given letter passes through the plugboard to output the final ciphered letter.  

In the real code, the `cipher_text` method calls on `cipher_letter` every iteration as it loops through the inputted plaintext. I can also log rotor state changes and their respective letter translation as the loop runs. 

# Enigma Simulation

In [ ]:
# -----------
# ENIGMA
# -----------
print("-----------")
print("ENIGMA")
print("-----------")

# creating sender machine
enigma_machine_sender = EnigmaMachine()
enigma_machine_sender.load_config_dict(config_dict=enigma_config)

# text to cipher
text = "THISISMYGRADETENWORLDWARTWOPROJECTONTHEENIGMAANDBOMBEMACHINESTHATSHAPEDTHEBATTLEOFTHEATLANTIC"
print("Clear text:")
print(text + "\n")

# cipher it
ciphered_text = enigma_machine_sender.cipher_text(text, print_display=True)
print("Ciphered Text:")
print(ciphered_text + "\n")

-----------
ENIGMA
-----------
Clear text:
THISISMYGRADETENWORLDWARTWOPROJECTONTHEENIGMAANDBOMBEMACHINESTHATSHAPEDTHEBATTLEOFTHEATLANTIC

ROTOR PANEL: 	 [16, 21, 6]
CIPHER: 	 T -> J

ROTOR PANEL: 	 [16, 21, 7]
CIPHER: 	 H -> S

ROTOR PANEL: 	 [16, 21, 8]
CIPHER: 	 I -> G

ROTOR PANEL: 	 [16, 21, 9]
CIPHER: 	 S -> Q

ROTOR PANEL: 	 [16, 21, 10]
CIPHER: 	 I -> L

ROTOR PANEL: 	 [16, 21, 11]
CIPHER: 	 S -> O

ROTOR PANEL: 	 [16, 21, 12]
CIPHER: 	 M -> R

ROTOR PANEL: 	 [16, 21, 13]
CIPHER: 	 Y -> V

ROTOR PANEL: 	 [16, 21, 14]
CIPHER: 	 G -> N

ROTOR PANEL: 	 [16, 21, 15]
CIPHER: 	 R -> P

ROTOR PANEL: 	 [16, 21, 16]
CIPHER: 	 A -> K

ROTOR PANEL: 	 [16, 21, 17]
CIPHER: 	 D -> R

ROTOR PANEL: 	 [16, 21, 18]
CIPHER: 	 E -> N

ROTOR PANEL: 	 [16, 21, 19]
CIPHER: 	 T -> C

ROTOR PANEL: 	 [16, 21, 20]
CIPHER: 	 E -> H

ROTOR PANEL: 	 [16, 21, 21]
CIPHER: 	 N -> U

ROTOR PANEL: 	 [16, 22, 22]
CIPHER: 	 W -> O

ROTOR PANEL: 	 [16, 22, 23]
CIPHER: 	 O -> U

ROTOR PANEL: 	 [16, 22, 24]
CIPHER: 	 

The code above uses `EnigmaMachine` to encode a string of plaintext that I wrote. As it loops through the text, we see the translations happening in real time, alongside the rotor panel. If you go through a few of the iterations, you may notice that the center roter shifts every time the right rotor reaches position 22. 

In [ ]:
new_turnover = (ALPHABET.index('R') - ALPHABET.index('V')) % 26
print("Turnover of rightmost Rotor I: Position {} at Letter {}".format(new_turnover, ALPHABET[new_turnover])) 

Turnover of rightmost Rotor I: Position 22 at Letter W


If we recall the ring setting and turnover point of rotor I, we would know that the new turnover point is indeed at position 22, specifically when the rotor reaches the letter W.

In [ ]:
# creating reciever machine
enigma_machine_sender = EnigmaMachine()
enigma_machine_sender.load_config_dict(config_dict=enigma_config)

# decipher the message
deciphered_text = enigma_machine_sender.cipher_text(ciphered_text, print_display=True)
print("Deciphered Text:")
print(deciphered_text + "\n")

ROTOR PANEL: 	 [16, 21, 6]
CIPHER: 	 J -> T

ROTOR PANEL: 	 [16, 21, 7]
CIPHER: 	 S -> H

ROTOR PANEL: 	 [16, 21, 8]
CIPHER: 	 G -> I

ROTOR PANEL: 	 [16, 21, 9]
CIPHER: 	 Q -> S

ROTOR PANEL: 	 [16, 21, 10]
CIPHER: 	 L -> I

ROTOR PANEL: 	 [16, 21, 11]
CIPHER: 	 O -> S

ROTOR PANEL: 	 [16, 21, 12]
CIPHER: 	 R -> M

ROTOR PANEL: 	 [16, 21, 13]
CIPHER: 	 V -> Y

ROTOR PANEL: 	 [16, 21, 14]
CIPHER: 	 N -> G

ROTOR PANEL: 	 [16, 21, 15]
CIPHER: 	 P -> R

ROTOR PANEL: 	 [16, 21, 16]
CIPHER: 	 K -> A

ROTOR PANEL: 	 [16, 21, 17]
CIPHER: 	 R -> D

ROTOR PANEL: 	 [16, 21, 18]
CIPHER: 	 N -> E

ROTOR PANEL: 	 [16, 21, 19]
CIPHER: 	 C -> T

ROTOR PANEL: 	 [16, 21, 20]
CIPHER: 	 H -> E

ROTOR PANEL: 	 [16, 21, 21]
CIPHER: 	 U -> N

ROTOR PANEL: 	 [16, 22, 22]
CIPHER: 	 O -> W

ROTOR PANEL: 	 [16, 22, 23]
CIPHER: 	 U -> O

ROTOR PANEL: 	 [16, 22, 24]
CIPHER: 	 M -> R

ROTOR PANEL: 	 [16, 22, 25]
CIPHER: 	 A -> L

ROTOR PANEL: 	 [16, 22, 0]
CIPHER: 	 Z -> D

ROTOR PANEL: 	 [16, 22, 1]
CIPHER: 	 J 

To validate that I indeed created an Enigma machine, I tested for reciprocity. Above, I run the enciphered message on a new Enigma machine for the reciever with the same settings as the sender machine. When passing the crptic string through the `cipher_text` method, out came the original message!

## How hard is it to crack this Enigma machine?

The Enigma machine only uses 3 rotors, which representative of the model used before 1942, but even with this setup, the chances of cracking it based on chance reaches the absurd numbers. To figure out the exact number of possibilities, the math gets a little messy. Here are some good finds that explain the probability:  

- [Numberphile Youtube](https://www.youtube.com/watch?v=G2_Q9FoD-oQ&t=375s)
- [Mathematical Security](http://users.telenet.be/d.rijmenants/en/enigmatech.htm#:~:text=In%20total%2C%20this%20gives%3A%2060%20x%2017%2C576%20x%20676%20x%20150%2C738%2C274%2C937%2C250%20%3D%20107%2C458%2C687%2C327%2C250%2C619%2C360%2C000%20or%201.07%20x%201023)

In Summary, with a 3-rotor Enigma with 10 plugboard connections gave a total of 107,458,687,327,250,619,360,000 or 1.07 x 10^23 possibilities of configurations. 100,000 operators each checking one setting every second would take twice the age of the universe to break the code! Even with the different shortcuts that the cryptologists at Bletcheley Park discovered, you wouldn't be able to beat the 24 hour clock at which new settings would be used on the Enigma machines.  


# The Flaw that Broke the Enigma

![](https://drive.google.com/uc?export=view&id=1N0K37s5yR47LdqdZ7QrjOeWrsPF4T37E)

Ironically, the one of the major factors that made the Enigma machine attractive for wartime communication was what also lead to its demise: the reciprocity. For example, if A translates to B, B also translated to A on an Enigma with the same settings. This characteristic lead to its major flaw, which was that a letter could never translate into itself. 

This discovery by the ULTRA team at Bletcheley Park (BP) lead to the discovery of Cribs by Alan Turing. 




## What are Cribs, Loops, and Menus?

Cribs are a very simple but clever concept that lead to the ability to discover the settings of the Enigma. Turing and the team at BP realized that certain words can be guessed based on the time of day. E.g. the word 'Weather' would commonly appear in encrypted messages sent during the morning. Expected phrases like 'heil Hitler' would also regularly appear on reports. 

![](https://drive.google.com/uc?export=view&id=1Ngwsi_wyjGx98BJh8IQtuqlg_KDKGlL6)

Given the discovery that letters can't encrypt into itself, BP would essentially slide plaintext cribs across encrypted messages. They would then record the patterns between letters. For example, in the image above, we can see that A clealy encrypts to E and I encrypts to E. Based on this pattern, a Turing loop can be created:

![](https://drive.google.com/uc?export=view&id=1spj6if_oOgE7qIkfrYFLtfYxqzLG4vDH)

The loop above identifies at which positions each letter encode into each other. Using these loops, more elaborate menus were created that would highlight all possible connections between letters. 

![](https://drive.google.com/uc?export=view&id=1OaBsABYKumFWBZiCp16xr2CMv1FU6jPr)

These menus would then be used to exhaustively iterate over all possible core rotor settings using a series of hypotheses and contradictions. In essence, we know that there is a setting out of the 107,458,687,327,250,619,360,000 that satisfies all the transformations in the menu, so we can go through the rotor setting and validate that a setting works by assuming plugboard values. The process for the diagram above is described here [http://www.ellsbury.com/bombe1.htm]:

"Let us test the hypothesis that E is steckered (plugged) to K. Now consider how E is transformed into A at relative position 23.

If E is steckered to K, K will be input to the scrambler at relative position 23 which will output some other letter v1. Since we know that E transforms into A at relative position 23, we know that A must be steckered to v1.

But if A is steckered to v1, v1 will be input to the scrambler at relative position 21 which will output some other letter v2. Since we know that A transforms into I at relative position 21, we know that I must be steckered to v2.

But if I is steckered to v2, v2 will be input to the scrambler at relative position 3 which will output some other letter v3. Since we know that I transforms into T at relative position 3, we know that T must be steckered to v3.

But if T is steckered to v3, v3 will be input to the scrambler at relative position 5.

Now consider the output of the scrambler at relative position 5 when it receives v3 as its input. Since we know that T transforms into E at relative position 5, we know that the output letter of the scrambler at relative position 5 must be steckered to E.

Suppose, when given v3 as its input, the scrambler at relative position 5 outputs J. By our original hypothesis E is steckered to K, but by our chain of implications E is steckered to J, so in this case E is steckered to both K and J. But the construction of the Enigma does not permit one letter to be steckered to two letters, so E cannot be steckered to both K and J, so our original hypothesis has resulted in an impossible consequence and so must be false.

But now suppose, when given v3 as its input, the scrambler at relative position 5 outputs K. Then by our original hypothesis E is steckered to K, and by our chain of implications E is steckered to K, so in this case E is steckered only to K, thus our original hypothesis has resulted in a consistent consequence and so may be true."

![](https://drive.google.com/uc?export=view&id=180U6AxNFVdH5oA9i6UcsGxwBgOpQ3g-q)

In the case that there was a contradiction, we would then try a different plugboard assumption from the letter E with the same rotor setting. After iterating through all 25 possible plugboard connections with the letter E, we would turn the rotors by a step and go through the whole process again. This would have been an eternally slow process to go through by hand, but 2 main discoveries made the lives of the BP cryptologists much easier:     
1. All plugboard connections used in the process of validating an assumption can be eliminated from the exhaustive search process once the assumption is proven wrong -> all the consecutive plugboard connections are the "fruit of a poisoned treee"
2. Validating all 26 plugboard connections for rotor setting instantaneously using electrically currents -> The Bombe




# Bombe Simulation

![](https://drive.google.com/uc?export=view&id=1QGbEDr-QJNE7DAGDUlBMQZCRDsSV-ajC)

The Bombe is simply several Enigma machines run in parallel, validating all possible settings using the rules and exploits discussed above. Turing realised that it was possible to represent the menus with electrical circuitry, which allowed to rapidly speed up the exhaustive search process. 

---



In [ ]:
from collections import defaultdict
from itertools import product

In [ ]:
class BombeMachine:

    def __init__(self, config_dict):
        """
        Initial configuration of the machine.
        """

        # load config file
        self.load_config_dict(config_dict)

        # create menu
        self.create_menu()


    def load_config_dict(self, config_dict):

          # crib
          self.crib_cipher = config_dict['crib']
          self.crib_plain = config_dict['cipher']

          # menu paths
          self.n_paths = config_dict['menus2check']
          self.paths = config_dict['paths']
          self.paths_input = self.paths[0][0]

          # rotors
          self.rotor_names = config_dict['rotor_combination']
          self.n_rotors = len(self.rotor_names)
          self.rotor_ring_settings = config_dict['ring_settings']
          self.reflector_name = config_dict['reflector']
            

    def create_menu(self):

        # model graph like: https://www.python.org/doc/essays/graphs/
        self.menu = defaultdict()

        for i, letter_cipher in enumerate(self.crib_cipher):
            
            letter_plain = self.crib_plain[i]

            # letter cipher -> letter plain
            if letter_cipher not in self.menu:
                links_dict = defaultdict()
                links_dict[letter_plain] = [i]
                self.menu[letter_cipher] = links_dict
            else: 
                links_dict = self.menu[letter_cipher]
                if letter_plain not in links_dict:
                    links_dict[letter_plain] = [i]
                else:
                    links_list = links_dict[letter_plain]
                    links_list.append(i)
                    links_dict[letter_plain] = links_list
                    
                self.menu[letter_cipher] = links_dict

            # letter plain -> letter 
            if letter_plain not in self.menu:
                links_dict = defaultdict()
                links_dict[letter_cipher] = [i]
                self.menu[letter_plain] = links_dict
            else: 
                links_dict = self.menu[letter_plain]
                if letter_cipher not in links_dict:
                    links_dict[letter_cipher] = [i]
                else:
                    links_list = links_dict[letter_cipher]
                    links_list.append(i)
                    links_dict[letter_cipher] = links_list
                    
                self.menu[letter_plain] = links_dict


    def add_contradiction(self, letter, letter_cipher):
        if letter not in self.contradictions:
            self.contradictions[letter] = set([letter_cipher])
        else: 
            plug_contradictions = self.contradictions[letter]
            plug_contradictions.add(letter_cipher)
            self.contradictions[letter] = plug_contradictions

        if letter_cipher not in self.contradictions:
            self.contradictions[letter_cipher] = set([letter])
        else: 
            plug_contradictions = self.contradictions[letter_cipher]
            plug_contradictions.add(letter)
            self.contradictions[letter_cipher] = plug_contradictions


    def run(self):

        # loop through all combinations of rotors
        rotor_positions_combinations = [''.join(i) for i in product(ALPHABET, repeat = 3)]

        # viable plugboard connections for each rotor combination
        self.possibilities = defaultdict()

        # initialize empty enigma machine
        enigma = EnigmaMachine()

        for rotor_positions in rotor_positions_combinations:

            #rotor_positions =  "DZR"
            #print("ROTOR POSITIONS: {}\n".format(rotor_positions))

            # impossible plugboard connections
            self.contradictions = defaultdict()

            # start making first plugboard guess
            for guess in ALPHABET:
                plugboard = defaultdict()
                plugboard[self.paths_input] = guess
                plugboard[guess] = self.paths_input
                plugboard_possible = True

                #print("Guess: P({})={}".format(self.paths_input, guess))

                # go through paths to check guess
                for path in self.paths:
                    
                    #print("Path: ", path)

                    for i in range( len(path) - 1 ): 

                        letter = path[i]
                        letter_connections = self.menu[letter]

                        letter_cipher = path[i+1]
                        letter_cipher_positions = letter_connections[letter_cipher] 

                        cipher_offset = letter_cipher_positions[0]
                        #TODO check when there is more than 1 connection

                        # load enigma config
                        enigma = EnigmaMachine()
                        enigma.load_config_bombe(self.n_rotors, self.rotor_names,
                                            self.rotor_ring_settings, rotor_positions, self.reflector_name)

                        enigma.step_rotors()
                        for co in range(cipher_offset):
                            #print display
                            '''
                            panel = "Display: "
                            for rotor_name in enigma.rotor_names:
                                rotor = enigma.rotors[rotor_name]
                                display = (rotor.position + rotor.ring_setting) % N_LETTERS
                                panel += "{}({}) ".format(ALPHABET[display], display)
                            print(panel)
                            '''
                            enigma.cipher_letter_bombe('X')
                            enigma.step_rotors()

                        if ((letter in plugboard) and plugboard_possible): 
                            plug_letter = plugboard[letter]
                            plug_letter_cipher = enigma.cipher_letter_bombe(plug_letter)

                            # P(letter_cipher) = S_pos( P(letter) )
                            '''
                            print("--")
                            print("{} = P( S_{}( P({}) ) )".format(letter_cipher, cipher_offset, letter))
                            print("P({}) = S_{}( P({}) ) = S_{}( {} )".format(letter_cipher, cipher_offset, letter, cipher_offset, plug_letter))
                            print("=> P({}) = {}".format(letter_cipher, plug_letter_cipher))
                            print(plug_letter_cipher)
                            '''

                            if letter_cipher in self.contradictions:

                                plug_contradictions = self.contradictions[letter_cipher]

                                if plug_letter_cipher in plug_contradictions:
                                    
                                    # merge with contradictions dictionary
                                    for plug in plugboard:
                                        plug_cipher = plugboard[plug]
                                        self.add_contradiction(plug, plug_cipher)

                                    # not worth keep checking on an inconsistency
                                    plugboard_possible = False
                                    '''
                                    print("\nContradiction previously found!")
                                    print("P({}) = {} causes a contradiction\n".format(letter_cipher, 
                                    plug_letter_cipher))
                                    '''
                                    break
                            
                            if letter_cipher in plugboard:

                                if (plugboard[letter_cipher] == plug_letter_cipher):

                                    # found end of loop
                                    '''
                                    print("\nEnd of loop reached:")
                                    print("{} = P({}) = {} means path finished\n".format(plugboard[letter_cipher], letter_cipher, plug_letter_cipher))
                                    '''
                                    break
                                else:
                                    # merge with contradictions dictionary
                                    self.add_contradiction(letter_cipher, plug_letter_cipher)
                                    for plug in plugboard:
                                        plug_cipher = plugboard[plug]
                                        self.add_contradiction(plug, plug_cipher)

                                    # not worth keep checking on an inconsistency
                                    plugboard_possible = False
                                    '''
                                    print("\nInconsistency found:")
                                    print("{} = P({}) = {} is not possible\n".format(plugboard[letter_cipher], letter_cipher, plug_letter_cipher))
                                    '''
                                    break
                            else: 
                                plugboard[letter_cipher] = plug_letter_cipher
                                plugboard[plug_letter_cipher] = letter_cipher
                    
                    if not plugboard_possible:
                        # avoid path because no assumptions can be made
                        break

                # Check if it found a good candidate and add it to the list
                if plugboard_possible:
                    
                    # TODO: check with the candidate plugboard letters if 
                    # another part of the message is decoded

                    tuple_plugboard = tuple(sorted(plugboard.items()))
                    if tuple_plugboard not in self.possibilities:
                        self.possibilities[tuple_plugboard] = [rotor_positions]
                    else:
                        self.possibilities[tuple_plugboard].append(rotor_positions)
                    '''
                    print("ROTOR POSITIONS: {}\n".format(rotor_positions))
                    print("Possible plugboard with guess P({})={}:".format(self.paths_input, guess))
                    print(tuple_plugboard)
                    print()
                    '''
                    
                #print("Moving to next guess...\n")

        '''
        print("POSSIBLE PLUGBOARDS:\n")
        total_possible_plugboards = 0
        for frozen_plugboard in self.possibilities:
            rotor_positions = self.possibilities[frozen_plugboard]
            n_possible_plugboards = len(rotor_positions)
            total_possible_plugboards += n_possible_plugboards
            print("{}: {}".format(frozen_plugboard, n_possible_plugboards))
            for rp in rotor_positions:
                print("- ", rp)
            print()

        print("Total possible plugboards + rotor positions: ", total_possible_plugboards) #692
        print("Different possible plugboards: ", len(self.possibilities.keys())) #624
        '''

        return self.possibilities


In [ ]:
bombe_config = {
    'crib':   'THISISMYGRADE',
    'cipher': 'JSGQLORVNPKRN',
    'menus2check': 7,
    'paths': ['LIG', 'GNE', 'HSO', 'IQ', 'SC', 'EN', 'RD'],
    'rotor_combination': ['V', 'IV', 'I'],
    'ring_settings': ['N', 'D', 'V'],
    'reflector': 'C'
}

In [ ]:
print("-----------")
print("BOMBE")
print("-----------")

bombe_machine = BombeMachine(bombe_config)
possible_plugboards = bombe_machine.run()
n_possible_plugboards = len(possible_plugboards.keys())

print("Different possible plugboards: " + str(n_possible_plugboards))
for frozen_plugboard in possible_plugboards:
  print(frozen_plugboard)
  rotor_positions = possible_plugboards[frozen_plugboard]
  print("\n- " + str(rotor_positions))

# *WORK IN PROGRESS* 